#### **REGULARIZATION**

In [6]:
%pip install pandas
%pip install seaborn
%pip install numpy
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.set_printoptions(precision=3, suppress=True)

In [9]:
# load the data and define some useful plotting functions.

np.random.seed(72018)



def to_2d(array):
    return array.reshape(array.shape[0], -1)


    
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

In [11]:
# load the data

%pip install requests

import requests

def download(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)

path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/boston_housing_clean.pickle"

# Download the dataset
download(path, "boston_housing_clean.pickle")

# Import pandas library
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [12]:
with open('boston_housing_clean.pickle', 'rb') as to_read:
    boston = pd.read_pickle(to_read)
boston_data = boston['dataframe']
boston_description = boston['description']

# show the first 5 rows using dataframe.head() method
print("The first 5 rows of the dataframe") 
boston_data.head()

The first 5 rows of the dataframe


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

In [13]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [14]:
# Import, fit, and transform using StandardScaler

from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

In [15]:
# Confirm standard scaling

a = np.array([[1, 2, 3], 
              [4, 5, 6]]) 
print(a) # 2 rows, 3 columns

[[1 2 3]
 [4 5 6]]


In [16]:
a.mean(axis=0) # mean along the *columns*

array([2.5, 3.5, 4.5])

In [17]:
a.mean(axis=1) # mean along the *rows*

array([2., 5.])

In [18]:
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)

True

In [19]:
# Coefficients with and without scaling

from sklearn.linear_model import LinearRegression

In [20]:
lr = LinearRegression()

y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [21]:
lr.fit(X, y)
print(lr.coef_) # min = -18

[ -0.107   0.046   0.021   2.689 -17.796   3.805   0.001  -1.476   0.306
  -0.012  -0.953   0.009  -0.525]


In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [24]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

[-0.92   1.081  0.143  0.682 -2.06   2.671  0.021 -3.104  2.659 -2.076
 -2.062  0.857 -3.749]


In [25]:
# what is the most "impactful" feature (this is intended to be slightly ambiguous)? 
# "In what direction" does it affect "y"?


pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)

,0,1
12,LSTAT,-3.748680
7,DIS,-3.104448
9,TAX,-2.075898
10,PTRATIO,-2.062156
4,NOX,-2.060092
0,CRIM,-0.920411
6,AGE,0.021121
2,INDUS,0.142967
3,CHAS,0.682203
11,B,0.856640


In [26]:
# Lasso with and without scaling

from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

In [27]:
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)

In [28]:
X_pf_ss = s.fit_transform(X_pf)

In [29]:
# LASSO

las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

array([-0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.991,  0.   , -0.   , -0.   ,  0.   , -0.   ,
        0.068, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,
       -0.   , -0.   , -0.   , -0.05 , -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   , -0.   ,  3.3  , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.42 , -3.498, -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   , -0.   , -0.   , -0.   , -0.146, -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ])

Compare

*   Sum of magnitudes of the coefficients
*   Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

*   Which do you expect to have greater magnitude?
*   Which do you expect to have more zeros?

In [30]:
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

sum of coefficients: 26.172415115426773
number of coefficients not equal to 0: 23


In [31]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())

sum of coefficients: 8.47240522776016
number of coefficients not equal to 0: 7


Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```

In [32]:
from sklearn.metrics import r2_score
r2_score(y,las.predict(X_pf_ss))

0.7207000461229028

In [36]:
# Train/Test Split

from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [38]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_test, y_pred)

0.6780325981174933

In [39]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_test, y_pred)

0.799926134284606

In [40]:
# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_test, y_pred))

r2 score for alpha = 0.001: 0.8847893236874315


In [41]:
# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_test,y_pred_lr))

r2 score for Linear Regression: 0.8689110469231036


In [42]:
# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())

Magnitude of Lasso coefficients: 435.57232290439254
Number of coeffients not equal to 0 for Lasso: 90
Magnitude of Linear Regression coefficients: 1183.8918138663455
Number of coeffients not equal to 0 for Linear Regression: 104


In [43]:
# L1 vs. L2 Regularization

from sklearn.linear_model import Ridge

In [44]:
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_

array([  7.692,   9.889, -25.005,   5.31 ,  -2.61 ,  14.958,  22.314,
       -22.858,  27.765,  -1.565,  17.077,  21.858,  11.574,   1.05 ,
         0.431,  13.786,   1.814,  -8.345,   4.994,  -3.564,  -3.436,
       -16.336,  -7.047,   6.609,  -1.477,   4.686,  -1.304,  -0.059,
        -0.304, -12.842,   1.977,   1.081,  -0.676,  -1.079,   4.492,
        -4.267,   4.675,  -1.281,   8.666,  -0.274,  -8.12 ,  11.783,
         6.538,   1.323,   2.06 ,   0.899,   1.789,   4.744,  -4.664,
         5.31 ,  -3.236,  -8.668,   0.973,   1.136,   0.29 ,  -1.631,
        -2.926,   2.923,  -0.735,  11.896,   0.754,  -7.531,  18.306,
       -22.168,  35.651, -22.381,  -7.378,   1.732,   4.068, -12.254,
        -3.721,  -5.537, -16.169,  -5.872,  -4.515,  -9.903,   0.942,
        -0.095,  17.18 , -14.268,  -2.904,  -2.824,  -5.418,  11.908,
         0.065,  -4.433,  -5.679,  -3.515,   0.822, -29.082,  49.345,
       -21.72 ,  -1.428,  -8.864, -15.682,  11.982,  -0.818,   2.924,
        -0.701,  -4.

In [45]:
las001 # same alpha as Ridge above

,alpha,0.001
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,100000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [46]:
las001.coef_

array([  0.   ,   0.   , -17.008,   2.592,   0.   ,  13.431,  10.062,
       -19.542,   9.237,   0.   ,   6.159,  17.087,  11.486,   1.208,
         0.219,  10.827,   2.185,  -7.106,   4.312,  -1.526,  -2.092,
        -9.669,   0.   ,   0.   ,  -1.177,   3.865,   0.378,   0.194,
        -0.296,  -3.478,   0.291,   0.72 ,  -0.795,  -0.74 ,   2.407,
        -0.892,   2.835,  -0.984,   3.864,  -0.963,   6.914,   6.2  ,
         4.197,   0.892,   2.028,   2.608,  -3.975,   2.574,  -4.568,
         2.086,  -2.   ,  -7.412,   1.604,   1.676,  -1.248,  -0.   ,
        -0.   ,   2.576,  -0.851,   3.206,  -0.   ,  -2.429,   9.276,
        -7.056,   0.   ,  -5.955,  -5.326,   0.531,   3.344,  -7.575,
        -0.   ,  -6.762, -10.2  ,  -6.428,  -2.835,  -9.561,   0.288,
         0.549,  11.094,  -6.731,  -1.161,  -2.301,  -3.555,   9.12 ,
        -0.014,  -2.384,  -2.042,  -1.691,   0.798, -20.822,  27.09 ,
         0.   ,  -1.527,  -6.793,  -5.084,  15.577,   0.   ,  -0.   ,
         0.569,  -3.

In [47]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

792.8673755104307
435.57232290439254
104
90


In [48]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

0.9076091395029262
0.9103503442034401


In [49]:
# scale

X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.6982083583132748

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [52]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_test, y_pred)

0.6982083583132748